In [ ]:
from diffusion import DiffusionEquation
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
import os
import imageio.v2 as imageio

In [8]:
def create_gif(folder, gif_name, fps=10):
    filenames = glob(os.path.join(folder, "*.png"))
    filenames.sort()
    images = []
    for filename in filenames:
        images.append(imageio.imread(filename))
    imageio.mimsave(f"./figs/{gif_name}.gif", images, loop=0, fps=fps)

In [ ]:
def simulate_and_create_heat_map(
    folder: str,
    gif_name: str,
    simulation: DiffusionEquation,
    steps: int,
    save_every: int,
    fps: int = 10,
):

    # remove all files inside the folder
    for f in glob(os.path.join(folder, "step*.png")):
        os.remove(f)

    # Create figs folder
    os.makedirs(folder, exist_ok=True)

    for i in range(steps):
        if i % save_every == 0:
            result = simulation.concentration_matrix

            sns.heatmap(result, vmin=0, vmax=1.0)
            plt.savefig(os.path.join(folder, f"step_{i:05d}.png"))
            plt.close()

        simulation.omp_step()

    create_gif(folder, gif_name, fps)

In [ ]:
simulation = DiffusionEquation("../build/libDiffusionEquation.so", N=50)

# Create a big concentration square in the middle of the matrix
center = simulation.N // 2
for i in range(-3, 4):
    for j in range(-3, 4):
        simulation.set_initial_concentration({(center + i, center + j): 1.0})


simulate_and_create_heat_map(
    folder="../data/figs/heatmap_center",
    gif_name="simulation_center",
    steps=25000,
    save_every=250,
    simulation=simulation,
    fps=10,
)

In [ ]:
simulation = DiffusionEquation("../build/libDiffusionEquation.so", N=50)

# Create a X shape inside the matrix
for i in range(1, simulation.N - 1):
    N = simulation.N - 1
    simulation.set_initial_concentration({(i, i): 1.0})
    simulation.set_initial_concentration({(i, N - i): 1.0})
    simulation.set_initial_concentration({(N - i, N - i): 1.0})
    simulation.set_initial_concentration({(i, N - i): 1.0})

center = simulation.N // 2
for i in range(-3, 4):
    for j in range(-3, 4):
        simulation.set_initial_concentration({(center + i, center + j): 1.0})

simulate_and_create_heat_map(
    folder="../data/figs/heatmap_X",
    gif_name="simulation_X",
    steps=25000,
    save_every=100,
    simulation=simulation,
    fps=30,
)